@origin script link: [Keywords Tagging](https://aiprd-jupyter.cisco.com/user/hdpiopsdp/notebooks/HadoopDir/ai_xray/scripts/ds/related_bugs/Keywords%20Tagging.ipynb#Tags-Generation) 

@ text annotation tool: [Doccano](https://github.com/doccano/doccano)

In [17]:
import pandas as pd
import spacy
import random
import sys
import json
from spacy import displacy

In [18]:
pwd

'/home/jupyter/Juliazhu'

# Training the NER model

In [19]:
# JSON(Text-Labels) file exported from doccano
# convert exported json file to format that spcay can digest

#fo = open('/home/jupyter/sw_xray_analysis/confidence_survey/DATA/training_data_500.json', "r")
fo = open('/home/jupyter/Juliazhu/DATA/input/survey_annotated_data.json', "r")
lines = fo.readlines()

TRAIN_DATA = []
LABELS = ['Keywords', 'Products', 'Counterparty', 'Positive', 'Negative']

for line in lines:

    line = json.loads(line)
    if "labels" in line:
        line["entities"] = line.pop("labels")
    else:
        line["entities"] = []

    tmp_ents = []
    for e in line["entities"]:
        
        if e[2] in LABELS:
            tmp_ents.append(tuple([e[0], e[1], e[2]]))
        
        line["entities"] = tmp_ents

    if (len(line["text"]) > 5):
        
        #print(tuple([line["text"],  {'entities':tmp_ents}]))
        TRAIN_DATA.append(tuple([line["text"],  {'entities':tmp_ents}]))


In [20]:
TRAIN_DATA[:1]

[('"It is always a user issue.  It is  not easy to install, usually taking 6-8hrs, then you have to make it do what you want it do.  Meraki just works.  We need to make it "" just works"".  It is a heavy lift to make it truly production.     2.  Education of customers:  not enough  knowledge transfer to teach the customer to make it work.   3. commitment to get the existing features work and ready. 4. get the SE community.   Quality has improved, but we then usually break something.  "',
  {'entities': [(130, 136, 'Counterparty'),
    (16, 26, 'Keywords'),
    (195, 205, 'Keywords'),
    (280, 298, 'Keywords'),
    (343, 353, 'Keywords'),
    (426, 433, 'Keywords'),
    (36, 44, 'Negative'),
    (468, 473, 'Negative'),
    (438, 446, 'Positive'),
    (243, 252, 'Keywords'),
    (374, 382, 'Keywords')]})]

In [21]:
# TRAIN_DATA = [('update symmetric keys for c8k products + ASR sysDataList fix', {'entities': [(7, 21, "cisco-term"), (26, 29, "cisco-term"), (41, 44, "cisco-term"), (45, 56, "cisco-term")]}), 
#                 ('SLE :DLC not processed on BF case upgraded from CSL to SLE with PAK(CSLU mode)', {'entities': [(0, 3, "cisco-term"), (5, 8, "cisco-term"), (9, 22, "verb"), (26, 28, "cisco-term"), (48, 51, "cisco-term"), (55, 58, "cisco-term"), (64, 67, "cisco-term"), (68, 72, "cisco-term")]}), 
#                 ('SLE : CSL -SLE image test is failing as auto DLC is not initiating?', {'entities': [(0, 3, "cisco-term"), (6, 9, "cisco-term"), (11, 14, "cisco-term"), (15, 25, "cisco-term"), (29, 36, "cisco-term"), (45, 48, "cisco-term")]}), 
#                 ('need registry to skip feature request for CDNA in SLE', {'entities': [(5, 13, "cisco-term"), (22, 37, "cisco-term"), (17, 21, "verb"), (42, 46, "cisco-term"), (50, 53, "cisco-term")]}), 
#                 ('Curie Thallium crashed while SL registering after switching C-->A', {'entities': [(0, 5, "cisco-term"), (6, 14, "cisco-term"), (15, 22, "verb"), (29, 31, "cisco-term"), (32, 43, "verb")]}), 
#                 ('missed commit for CSCvq36980 on polaris_dev', {'entities': [(32, 43, "cisco-term"), (18, 28, "cisco-term")]}), 
#                 ('throughput is allowed even when license is not authorized in SLR mode for CSR?', {'entities': [(0, 10, "cisco-term"), (32, 39, "cisco-term"), (74, 77, "cisco-term"), (61, 64, "cisco-term")]}), 
#                 ('License Registration on the CSR fails intermittently', {'entities': [(0, 20, "cisco-term"), (28, 31, "cisco-term"), (32, 37, "verb")]}), 
#                 ('Curie/SDWAN: linux_iosd crash observed on sdwan image', {'entities': [(0, 5, "cisco-term"), (6, 11, "cisco-term"), (13, 23, "cisco-term"), (42, 53, "cisco-term"), (24, 29, "verb")]}), 
#                 ('Curie/SDWAN: Reva-NIM going out-of -service on sdwan image', {'entities': [(0, 5, "cisco-term"), (6, 11, "cisco-term"), (13, 21, "cisco-term"), (47, 58, "cisco-term"), (28, 43, "verb")]}), 
#                 ('Curie/SDWAN: device keep reloading if bootflash memory is lessthan 5GB', {'entities': [(0, 5, "cisco-term"), (6, 11, "cisco-term"), (25, 34, "verb"), (38, 47, "cisco-term")]}), 
#                 ('Curie/SDWAN: Prince module state showing as out-of-service on SDWAN image', {'entities': [(0, 5, "cisco-term"), (6, 11, "cisco-term"), (13, 19, "cisco-term"), (44, 58, "verb"), (62, 67, "cisco-term")]}), 
#                 ('Routing: SLR Exploitation Auth code can be used repeatedly after return code with factory-reset', {'entities': [(0, 7, "cisco-term"), (9, 12, "cisco-term"), (26, 35, "cisco-term"), (82, 95, "cisco-term")]})]



# TRAIN_DATA = [('what is the price of polo?', {'entities': [(21, 25, 'PrdName')]}), 
#               ('what is the price of ball?', {'entities': [(21, 25, 'PrdName')]}), 
#               ('what is the price of jegging?', {'entities': [(21, 28, 'PrdName')]}), 


TRAINING_ITERATION = 5
TRAINING_DATA = TRAIN_DATA

def train_spacy(data,iterations):
    TRAIN_DATA = data
    nlp = spacy.blank('en')  # create blank Language class
    #nlp = spacy.load('en')  # load Language class
    # create the built-in pipeline components and add them to the pipeline
    # nlp.create_pipe works for built-ins that are registered with spaCy
    if 'ner' not in nlp.pipe_names:
        ner = nlp.create_pipe('ner')
        nlp.add_pipe(ner, last=True)
       

    # add labels
    for _, annotations in TRAIN_DATA:
         for ent in annotations.get('entities'):
            ner.add_label(ent[2])

    # get names of other pipes to disable them during training
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
    with nlp.disable_pipes(*other_pipes):  # only train NER
        optimizer = nlp.begin_training()
        for itn in range(iterations):
            print("Statring iteration " + str(itn))
            random.shuffle(TRAIN_DATA)
            losses = {}
            for text, annotations in TRAIN_DATA:
                nlp.update(
                    [text],  # batch of texts
                    [annotations],  # batch of annotations
                    drop=0.2,  # dropout - make it harder to memorise data
                    sgd=optimizer,  # callable to update weights
                    losses=losses)
            print(losses)
    return nlp


prdnlp = train_spacy(TRAINING_DATA, TRAINING_ITERATION)

Statring iteration 0
{'ner': 876.9709626607478}
Statring iteration 1
{'ner': 409.2087106840477}
Statring iteration 2
{'ner': 323.4297826275687}
Statring iteration 3
{'ner': 392.0750346406912}
Statring iteration 4
{'ner': 372.49866494443194}


In [23]:
# sasve the model to disk
#prdnlp.to_disk('/home/jupyter/sw_xray_analysis/confidence_survey/DATA/output/cisco_survey_spacy_ner_model')

# Visualizing the keywords

In [24]:
text = "But dakan wei Google PS4 is starting from behind. dakan wei still like playing Nintendo Switch, The company made a late push into hardware, and Apple’s Siri, available on iPhones, and Amazon’s Alexa software, which runs on its Echo and Dot devices, have clear leads in consumer adoption."

doc = prdnlp(text)
#print('Entities', [(ent.text, ent.label_) for ent in doc.ents])
for ent in doc.ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)

Google PS4 14 24 Keywords
behind 42 48 Keywords
dakan 50 55 Keywords
Switch 88 94 Counterparty
late push 115 124 Keywords
Apple 144 149 Keywords
Amazon 184 190 Counterparty
Alexa software 193 207 Keywords
on 220 222 Counterparty
Echo 227 231 Keywords
Dot 236 239 Counterparty
clear leads 254 265 Keywords
adoption 278 286 Keywords


In [31]:
#prdnlp = spacy.load("gs://swxray/DATA/Grace/DNAC/bert model/cisco_survey_spacy_ner_model/")

In [27]:
prdnlp = spacy.load("/home/jupyter/sw_xray_analysis/general_cisco_keywords_extraction/DATA/cisco_survey_spacy_ner_model")

Input text data:

In [28]:
text = "DNAC's primary value (for wireless)  need to be rock solid, don't need all the other less imporatn features to be good from the get-go.    2.  From the Cost and  use case,  the wireless assurance use case is the most imporant.   SDA for wired.      Wireless assuance needs to be solid.   Meraki workflow should be the example to follow.  Focus on a few things and make them simpler, rather than everything.     Because the products that do not work, our own reputation as specialists are also on the line."


In [29]:
# https://spacy.io/usage/visualizers#ent
# colors = {"KEYWORDS": "linear-gradient(90deg, #aa9cfc, #fc9ce7)"}

colors = {"KEYWORDS":     "#D8E0BB",
         "PRODUCTS":      "#35BCEC",
         "COUNTERPARTY":  "#F6D55C",
         "POSITIVE":      "#4BFA6B",
         "NEGATIVE":      "#FA2A24"}


options = {"colors": colors}


doc = prdnlp(text)
displacy.render(doc, style="ent", jupyter=True, options=options)
